In [65]:
import pandas as pd

df=pd.read_csv('./Latest_ETM_Data/Final_Ticket_data_2024-07-03.csv')
df.columns

Index(['id', 'depot_code', 'depot', 'fleet_no', 'route_service_type',
       'trip_no', 'ticket_no', 'date', 'time', 'ticket_type', 'source',
       'source_stop_no', 'source_zonal_centroid_number', 'destination',
       'destination_stop_no', 'destination_zonal_centroid_number',
       'passenger_count', 'luggage_count', 'passenger_amount',
       'luggage_amount', 'driver_no', 'conductor_no', 'waybill_no',
       'schedule_no', 'payment_type', 'direction', 'Trip_Origin',
       'Trip_Destination', 'total_ticket_amount', 'KM', 'duty_date',
       'Direction_route'],
      dtype='object')

In [66]:
passenger_counts = df['passenger_count'].sum()
print("Total passenger count :", passenger_counts)
zero_value_count = df[df['source'] == 'ZERO VALUE TICKET'].shape[0]
print("Zero value ticket rows :", zero_value_count )

Total passenger count : 2232820
Zero value ticket rows : 863553


In [67]:
max_value = df['source_zonal_centroid_number'].max()
print(f"Maximum value of source_zonal_centroid_number is: {max_value}")

max_dest = df['destination_zonal_centroid_number'].max()
print(f"Maximum value of destination_zonal_centroid_number is: {max_dest}")
#3039

Maximum value of source_zonal_centroid_number is: 3039.0
Maximum value of destination_zonal_centroid_number is: 3039.0


In [52]:
#multiply_factor=

In [53]:
# Check for duplicate rows
duplicate_rows = df[df.duplicated()]
print("Number of duplicate rows:", duplicate_rows.shape[0])



Number of duplicate rows: 0


In [60]:
all_zonal_centroids = range(1, 3040)

# Create all possible source-destination pairs
all_pairs = pd.MultiIndex.from_product(
    [all_zonal_centroids, all_zonal_centroids],
    names=['source_zonal_centroid_number', 'destination_zonal_centroid_number']
)

# Group and sum passenger counts, then reindex to include all pairs
grouped = df.groupby(['source_zonal_centroid_number', 'destination_zonal_centroid_number'])['passenger_count'].sum()
od_matrix = grouped.reindex(all_pairs, fill_value=0).unstack(fill_value=0)

# Add a column for the row sums
od_matrix['Boarding'] = od_matrix.sum(axis=1)
od_matrix.loc['Alighting'] = od_matrix.sum(axis=0)

od_matrix

destination_zonal_centroid_number,1,2,3,4,5,6,7,8,9,10,...,3031,3032,3033,3034,3035,3036,3037,3038,3039,Boarding
source_zonal_centroid_number,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,6,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,35081
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,30,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3036,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3037,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3038,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
import pandas as pd

all_zonal_centroids = range(1, 3039)

# Create all possible source-destination pairs
all_pairs = pd.MultiIndex.from_product(
    [all_zonal_centroids, all_zonal_centroids],
    names=['source_zonal_centroid_number', 'destination_zonal_centroid_number']
)

combined_grouped = pd.Series(0, index=all_pairs, dtype='int64')

# Loop through each date and aggregate passenger counts
for day in range(1, 2):  # From 01 to 07
    date = f"2024-07-{day:02d}"
    file_path = f"./Latest_ETM_Data/Final_Ticket_data_{date}.csv"
    
    # Load the daily data
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        continue
    
    # Group passenger counts for this day's data
    daily_grouped = df.groupby(
        ['source_zonal_centroid_number', 'destination_zonal_centroid_number']
    )['passenger_count'].sum()
    
    # Add to the combined grouped data
    combined_grouped = combined_grouped.add(daily_grouped, fill_value=0)

# Construct the OD matrix from the combined passenger counts
od_matrix = combined_grouped.reindex(all_pairs, fill_value=0).unstack(fill_value=0)

# Add a column for the row sums (Total Passengers)
od_matrix['Total Passengers'] = od_matrix.sum(axis=1)

od_matrix.head(10)

destination_zonal_centroid_number,1,2,3,4,5,6,7,8,9,10,...,3030,3031,3032,3033,3034,3035,3036,3037,3038,Total Passengers
source_zonal_centroid_number,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35081.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,894.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,140.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0
